# **Lets create the big dataframe for blood**


**strucutre of this notebook :**
1. **Blood sites on blood data**
    - 1.1 imports 
    - 1.2 get all mutant sites 
    - 1.3 append non-mut sites --> no overlap with mutant sites (takes 20mins) 
    - 1.4 basic declarations for the loop --> nte that still not using all tracks (ie hapmap ) 
    - 1.5 diagnostics (Xed out for script: was an interactive chunk) 
    - 1.6 big daddy loop 
    - 1.7 loop print error lists 

In [107]:
dnase_file = "data/blood/track_data/DNAse/wgEncodeUwDnaseSeqRawSignalRep1Gm12878.bed.gz"
h3k27ac_file = "data/blood/track_data/H3k27/wgEncodeBroadChipSeqSignalGm12878H3k27ac.bed.gz"
h3k4me1_file = "data/blood/track_data/H3k4me1/wgEncodeBroadChipSeqSignalGm12878H3k4me1.bed.gz"
h3k4me3_file = "data/blood/track_data/H3k4me3/wgEncodeBroadChipSeqSignalGm12878H3k4me3.bed.gz"
# methylationCHG_file = "bgzip  data/blood/track_data/methylation/methylation_CHG.bed.gz"
# methylationCHH_file = "bgzip data/blood/track_data/methylation/methylation_CHH.bed.gz"
# methylationCpG_file = "bgzip data/blood/track_data/methylation/methylation_CpG.bed.g
laminB1_file = "data/global/track_data/laminB1/hg18.laminB1.bed.gz"
recombination_file = "data/global/track_data/recombination/recombination.bed.gz"
repeats_file = "data/global/track_data/repeats/repeats.bed.gz"
replication_file = "data/global/track_data/replication/replication.bed.gz"
# phastcons_fileLoc = "data/global/track_data/phastcons/"

#dictionry where i specify which col contains the information in the datafile , 0 indexed 
tracksColFile_dict = {"H3k27":[4,"../../data/blood/track_data/H3k27/wgEncodeBroadChipSeqSignalGm12878H3k27ac.bed.gz"], 
                      "H3k4me1":[4,"../../data/blood/track_data/H3k4me1/wgEncodeBroadChipSeqSignalGm12878H3k4me1.bed.gz"], 
                       "H3k4me3":[4,"../../data/blood/track_data/H3k4me3/wgEncodeBroadChipSeqSignalGm12878H3k4me3.bed.gz"], 
                      "DNAse":[4,"../../data/blood/track_data/DNAse/wgEncodeUwDnaseSeqRawSignalRep1Gm12878.bed.gz"], 
                      "laminB1":[4,"../../data/global/track_data/laminB1/hg18.laminB1.bed.gz"], 
                      "recombination": [4,"../../data/global/track_data/recombination/recombination.bed.gz"],
                       "Repeats":["binary", "../../data/global/track_data/repeats/repeats.bed.gz"],
                      "Replication":[3,"../../data/global/track_data/replication/replication.bed.gz"]} #remove nucmec/dennis, phastcons, 

## **1.1 imports** 

In [74]:
import random
import pandas as pd
import numpy as np 
from numpy.random import choice
import matplotlib.pyplot as plt
import collections
from Bio import AlignIO
import pysam 
from tqdm import tqdm 
from datetime import datetime
import gzip

## **1.2  mutant sites list** 

In [33]:
mutations_lines = open('../../data/blood/mutations/blood_mutations_hg18_sorted_tabdelim.bed').readlines()

In [34]:
mutations_lines[1]

'chr1\t1212518\t1212519\tID_1_individual_2_single-cell_1\tC\tT\t0.59\tSCNN1D\texon\tY=>Y\t-\thuman\tfemale\t39\tblood(hematopoietic_stem/progenitor_cells,HSPCs)\twhole_genome_sequencing(single-stem-cell_clonal_culture)\tbone_marrow_mononuclear_cells\n'

In [16]:
#create a dictionary where each chrom key will have a n empty list  
muts_bychrom_dict = {}
for x in range(1,23): 
    key_string = 'chr{n}'.format(n=x)
    muts_bychrom_dict[key_string] = []
    
#fill each chromosome's empty list  with the sites for that chrom 
non_chrnMuts = []#create list of chrom names that dont belong to chrN format --> disgnostic 
for x in range(1,len(mutations_lines)): 
    chrom_mut = mutations_lines[x].split()[0]
    mut_startSite = mutations_lines[x].split()[1]
    if chrom_mut in muts_bychrom_dict.keys():                  #controlling for chrX/chrY
        muts_bychrom_dict[chrom_mut].append(mut_startSite)
    else: 
        non_chrnMuts.append(chrom_mut)

        
#testing making usre the only sites that dont make it are sex chromosome mutations 
print(len(non_chrnMuts),"  non chrN muts (ommited) from these lables:", np.unique(non_chrnMuts))

#add the sites infro from file 
sites = []#sites = list of sites 
for chrom_key in muts_bychrom_dict.keys(): 
    for mutation_element in muts_bychrom_dict[chrom_key]: 
        sites.append([chrom_key, int(mutation_element),1]) #the 1 is for mutation status column. 1 = yes 
        
print("number included mutations = " +str(len(sites)))

387   non chrN muts (ommited) from these lables: ['chr17_random' 'chr21_random' 'chrX' 'chrY']
number included mutations = 8776


## **1.3 append non-mutations to sites list** 

In [17]:
#get chrom length information so I can perform weighted choice for non-mut site selection 
ChromLengths = pd.read_csv('../../data/global/sequence/hg38_chromosomelengths.csv') #read in the csv file of hg38 chrom lengths I found on the internets 
total_length=0 #lets sum (get the total length) 
for length in list(ChromLengths.Length): 
    total_length+=int(length.replace(",",""))

#build dictionary to store porbability 
dict_lengths = {}#creat emepty dictionary 
for x in range (0,22): 
        tmp_index = x +1
        length = str(ChromLengths[x:x+1]).split()[4]
        length = length.replace(",", "")
        length = int(length)
        dict_lengths["chr"+str(tmp_index)] = length

#make the porbability of choosing a chrom based on length 
list_0chroms = ['chr01', 'chr02', 'chr03','chr04','chr05','chr06','chr07','chr08','chr09','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22']
list_chroms = ['chr' + str(i) for i in range(1, 23)]
list_chrom_probabilities = []
for chrom in list_chroms: 
    list_chrom_probabilities.append(dict_lengths[chrom]/total_length)

#maing sure the probability list sums to one (arbitrarily add the rounding error to chr1)
list_chrom_probabilities[0] = list_chrom_probabilities[0]+1-sum(list_chrom_probabilities) # adds the 0.00000001 left from rounding errors to the chr1 so sum adds perfectly to 1. 
assert(sum(list_chrom_probabilities)==1)

#perfrom the non-mutant site draw 
number_nonmuts = len(sites)
chrom_draw = choice(list_chroms, number_nonmuts,p=list_chrom_probabilities)

#visualize the draw: 
#print(collections.Counter(draw))

#make the sites list with the chr# and site 
for chrom in tqdm(chrom_draw): 
    site_draw = random.randrange(1,dict_lengths[chrom])
    if [chrom,site_draw] not in sites: 
        sites.append([chrom,site_draw,0]) # the 0 if for the mutation status column. 0 = no 
# print(len(sites))
#haha there was one mutation overlap!! 

100%|████████████████████████████████████████████████████████████████████████████| 8776/8776 [00:01<00:00, 6469.53it/s]


## **1.4 basic declarations for the loop** 

**tissue specific declarations (may chnage b/t somatic models )** 

In [62]:
mutations_df = pd.read_table('../../data/blood/mutations/blood_mutations_hg18_sorted_tabdelim.bed', header = None)

In [66]:
#rename header for easier access down the line 
#replace space with underscore, / with Or, and Age(yars) with AgeInYears
mutations_df.columns = ['chromosome',  'start',  'fake_end', 'file_ID',  'Reference_allele', 'Variant_allele', 'VAF', 'Gene_name', 'Region',
 'AA', 'COSMIC', 'Species', 'Gender', 'Age(years)', 'Tissue/Cell_type', 'Single-cell_genomics_biotechnology/Method', 'Control_sample/tissue']


mutations_list = mutations_df.start.values.tolist()  # may need to chnage the name ofwhich coloumn has the mutations 

**declarations that should stay the same for all tissues** 

In [72]:
#basic declarations (constants) 
list_tissueSpecific_tracks = ["Transcription", "DNAse", "H3k4me1", "H3k4me3", "H3k27"]

row =[] #final product that gets printed to file (after str conversion)

alignment_discordant = []                                    #used to collect all the instances where the hg18 fasta result doesnt match the mutation dataset "Ref" column 
tracks_NA_is_zero = ["Transcription", "laminB1", "Repeats", "H3k4me1", "H3k4me3", "H3k27", "DNAse"] #remove nucmec/dennis
list_no_seq_at_site = []
list_sites_outOfBufferRange = []

fastas_dict = {}   # creating dictionary with fasta alignment, length of seq, 
for chrom in list_chroms:
    filename_tmp = "../../data/global/sequence/{c}.fa.gz".format(c=chrom)
    fastas_dict[chrom] = []
    with gzip.open(filename_tmp, "rt") as handle:
        fastas_dict[chrom].append(AlignIO.read(handle,"fasta"))
        alignment_tmp = fastas_dict[chrom][0]
        fastas_dict[chrom].append(len(str(alignment_tmp[0].seq)))


#variable declaration  
list_of_surrounding_contexts = [0,100,10000] #note if you chnagee/ increase this, then you oncrese the buffer zone (not using sites in the buffer, len(dna)-max_distance )
distance_max = max(list_of_surrounding_contexts)

# **1.6 big daddy loop** 

**table creation notes** 
1. no overlapping non-muts sites 
2. sites are picked allover the genome (ind of buffer) but then sites outside buffer excluded from table 
    - can keep track of sites in "list_sites_outOfBufferrange
3. if the site was mutant, then triplet centre base chnages (not good!) 

In [ ]:
#code: 
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
with open('../../data/blood/predictorDf_{t}_updatedTriplet.txt'.format(t=timestamp), 'w') as f:                                                    # opening line to write to a file 
    
    header = "Chromosome"+ "\t"+"site"+"\t" +"triplet"+"\t"+"mutation_status"                        # creating the begining of the header 
    for trackname in tracksColFile_dict.keys():                                                     # the rest of the header is a function of tracks 
        for distance in list_of_surrounding_contexts:                                                # and distance (need a col for every track and for every distance value within ) 
            header = header + "\t"+str(trackname)+"-"+str(distance)
    for distance in list_of_surrounding_contexts:                                                    # creating the end of the header assoc with no track (the seqeunce at different 
        header = header + "\t"+"Apercent-"+str(distance)+ "\t"+"Gpercent-"+str(distance)+ "\t"+"Cpercent-"+str(distance)+ "\t"+"Tpercent-"+str(distance)   # distace values) 
    header = header +"\n"                                                                            # obviously needs to end with a \n 
    f.write(header)                                                                                  #write header to file 

    
    for site in tqdm(sites): 
        if site[1] <= distance_max or site[1]+distance_max >= fastas_dict[site[0]][1]:               # only use sites that will have values for site +- max distance (buffer). second element in fastas dict is the length 
            list_sites_outOfBufferRange.append(site)
        
        else: 
            row = []
            row.extend([site[0], site[1]])
            alignment = fastas_dict[site[0]][0]                                                       #create the alingment from the list of fastas 
            
            
            #makes the triplet depending on itf mutant or nah 
            if site[2] ==1:                                     #muts_by-chrom_dict is literally a dictionary containing list of sites that are mutations in that chrom. 
                mutation_row = mutations_df[(mutations_df.chromosome == site[0]) & (mutations_df.start == site[1])]  #get the row containing mut info out of the df 

                mutation_bp = mutation_row.Variant_allele.values[0]
                mutated_triplet = str(alignment[0,site[1]-1])+str(mutation_bp)+str(alignment[0,site[1]+1]).upper()

                old_bp = mutation_row.Reference_allele.values[0]
                old_triplet = str(alignment[0,site[1]-1])+str(old_bp)+str(alignment[0,site[1]+1]).upper()

                row.extend([mutated_triplet, 1])

                seq_triplet = str(alignment[0,site[1]-1:site[1]+2].seq)
                if old_triplet.upper() != seq_triplet.upper(): 
                    alignment_discordant.append(old_triplet+" "+str(mutation_row)+" "+str(sites.index(site))+" "+str(site))
                    
            else: 
                triplet= str(alignment[0,site[1]-1:site[1]+2].seq).upper()
                row.extend([triplet,0])
            
            #converting fro 0 to nomral chrom lists so can get correct files 
#             index_chromConversion = list_chroms.index(site[0])                                        #takes the first element of site (chr from tabix) and searches for the index within the list. 
#             chrom_forFiles = list_0chroms[index_chromConversion]                                      #used the previous index to get out the ch 0 number needed for the files : chr01 

            #track stuff 
            for trackname,track_val in tracksColFile_dict.items():                 
                
                filename = track_val[1]                   
                    
                for distance in list_of_surrounding_contexts: 
                    #track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                    if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]:                #if no value at that site 
                        if trackname in tracks_NA_is_zero: 
                            row.append(0)
                        else: 
                            row.extend(["NA"])                                                                                      

                    else:                                                   
                        track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                        multiple_values = []
                        if tracksColFile_dict[trackname][0] == 4:
                            for element in track_output: 
                                multiple_values.append(float(element.split()[4]))
                            #print(multiple_values)
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 3: 
                            for element in track_output: 
                                multiple_values.append(float(element.split()[3]))
                            #print(multiple_values)
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 'binary': 
                            row.append(len(track_output))
                        else: 
                            print("ERROR: track coloumns not 4 or 5 or binary")
                            print(tracksColFile_dict[trackname])

            #sequence stuff                
            for distance in list_of_surrounding_contexts: 
                seq_around = str(alignment[0,site[1]-1:site[1]+1+1].seq)
                if seq_around != '': 
                    seq_around = str(alignment[0,site[1]-distance:site[1]+distance+1].seq)
                    Acount = seq_around.count('a')+seq_around.count("A")
                    Gcount = seq_around.count('g')+seq_around.count("G")
                    Ccount = seq_around.count('c')+seq_around.count("C")
                    Tcount = seq_around.count('t')+seq_around.count("T")
                    Apercent = Acount/len(seq_around)
                    Gpercent = Gcount/len(seq_around)
                    Cpercent = Ccount/len(seq_around)
                    Tpercent = Tcount/len(seq_around)
                    row.extend([Apercent, Gpercent, Cpercent, Tpercent])
                else: 
                    row.extend(['NA','NA','NA','NA'])
                    list_no_seq_at_site.append(site)
            #print(header
            #print(row)          
            #print(len(row))
            row_string = str()
            for i in range(0,len(row)): 
                row_string = row_string+str(row[i])+"\t"
            row_string = row_string.rstrip("\t")
            row_string = row_string+"\n"
            f.write(row_string)
print(len(list_sites_outOfBufferRange))

  6%|█████▋                                                                                  | 1132/17552 [18:21<4:23:42,  1.04it/s]

## ** 1.7 printing error output of the model ** 

In [114]:
print(len(list_sites_outOfBufferRange))
print(len(alignment_discordant))
print(len( list_no_seq_at_site))

150
6602
0


In [ ]:
test

In [116]:
len(alignment_discordant)/len(sites)

0.3761394712853236